In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
from IPython import display

from deep_shadow import *
from utils import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Loading Dataset

In [3]:
cities = ['nyc']
dates = ['summer']

zoom = 16
shadow_path = 'data/shadows/'
height_path = 'data/heights_new/'

TILES_PER_CITY = 200
BATCH_SIZE = 1

In [5]:
train_dataset, test_dataset = get_train_test(
    height_path, shadow_path, cities, dates, zoom, TILES_PER_CITY, BATCH_SIZE)

### Latitude and date inclusion

In [6]:
lat, dat = False, False

### Store Model Results

In [12]:
def store_result(checkpoint_name, deep_shadow, test_dataset, lat, dat):
    df = pd.read_csv('data/compare_models/results.csv')
    rmses, maes, mses, ssims, sobels = get_metrics(test_dataset, deep_shadow.generator, latitude=lat, date=dat)
    df.loc[len(df.index)] = [checkpoint_name, np.average(maes), np.average(mses), np.average(rmses), np.average(ssims), np.average(sobels)] 
    df.to_csv('data/compare_models/results.csv', index=False)

    # print('MAE: %.4f' % np.average(maes), 'MSE: %.4f' % np.average(mses), 'RMSE: %.4f' % np.average(rmses), 'SSIM: %.4f' % np.average(ssims), 'Sobel: %.4f' % np.average(sobels))

    return

### Train models with different loss functions and save results

In [14]:
checkpoint_names = ['_l1', '_l2', '_l1_smooth_loss', '_ssim', '_sobel', '_ssim_with_sobel', '_ssim_l1_smooth_loss', '_ssim_l1_loss', '_ssim_sobel_plus_l1', '_berhu']
loss_functions = [[l1_loss], [l2_loss], [l1_smooth_loss], [ssim_loss], [sobel_loss], [ssim_loss, sobel_loss], [ssim_loss, l1_smooth_loss], [ssim_loss, l1_loss], [ssim_loss, sobel_loss, l1_loss], [berhu_loss]]

for i in range(len(checkpoint_names)):
    tf.keras.backend.clear_session()

    down_stack, up_stack = get_generator_arch('resnet9', attn=False)
    
    train_dataset, test_dataset = get_train_test(
    height_path, shadow_path, cities, dates, zoom, TILES_PER_CITY, BATCH_SIZE)

    checkpoint_name = 'test' + checkpoint_names[i]
    checkpoint_path = 'training_checkpoints/%s' % (checkpoint_name)
    deep_shadow = DeepShadow(512, 512, down_stack, up_stack, latitude=lat, date=dat, loss_funcs=loss_functions[i], type='unet', attention=False, model_name=checkpoint_name)
    deep_shadow.fit(checkpoint_path, train_dataset, test_dataset, 40000)

    deep_shadow.restore(checkpoint_path)
    store_result(checkpoint_name, deep_shadow, test_dataset, lat, dat)